In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import torch

import torch.nn as nn

from src.consts import *
from src.main import main, setup_torch, get_corpus
from src.model import RNNModel
from src.training import train, evaluate
from src.split_cross_entropy_loss import SplitCrossEntropyLoss

from notebooks.utils import summary, check_cuda_mem

In [3]:
use_data_paralellization = False

In [4]:
setup_torch()
device = torch.device("cuda" if USE_CUDA else "cpu")
corpus = get_corpus()

# TODO remove these two lines
assert len(corpus.dictionary) == 602755
assert corpus.valid.size()[0] == 11606861
assert corpus.train.max() < len(corpus.dictionary)
assert corpus.valid.max() < len(corpus.dictionary)
assert corpus.test.max() < len(corpus.dictionary)

ntokens = len(corpus.dictionary)
model = RNNModel(MODEL_TYPE, ntokens, EMBEDDINGS_SIZE, HIDDEN_UNIT_COUNT, LAYER_COUNT, DROPOUT_PROB,
                 TIED)
if use_data_paralellization or USE_DATA_PARALLELIZATION:
    model = CustomDataParallel(model)
else:
    model.to(device)
criterion = nn.CrossEntropyLoss()

summary(model, criterion)

RNNModel(
  (drop): Dropout(p=0.2)
  (encoder): Embedding(602755, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=602755, bias=True)
)

encoder.weight torch.Size([602755, 200])
rnn.weight_ih_l0 torch.Size([800, 200])
rnn.weight_hh_l0 torch.Size([800, 200])
rnn.bias_ih_l0 torch.Size([800])
rnn.bias_hh_l0 torch.Size([800])
rnn.weight_ih_l1 torch.Size([800, 200])
rnn.weight_hh_l1 torch.Size([800, 200])
rnn.bias_ih_l1 torch.Size([800])
rnn.bias_hh_l1 torch.Size([800])
decoder.weight torch.Size([602755, 200])
decoder.bias torch.Size([602755])

Total Parameters: 121,796,955


In [5]:
train(model, corpus, criterion, device)

INFO 2019-05-24 16:47:08,446: | epoch   1 |   200/23759 batches | lr 20.00 | ms/batch 170.86 | loss  9.71 | ppl 16518.85
INFO 2019-05-24 16:47:42,700: | epoch   1 |   400/23759 batches | lr 20.00 | ms/batch 171.27 | loss  8.27 | ppl  3900.39
INFO 2019-05-24 16:47:45,781: -----------------------------------------------------------------------------------------
INFO 2019-05-24 16:47:45,782: Exiting from training early


FileNotFoundError: [Errno 2] No such file or directory: 'models/trained_models/model-2019-05-24 16:46:34.273837.pt'

In [6]:
# timestamp = datetime.datetime.now()
# with open(MODEL_FILE_NAME.format(timestamp), 'wb') as f:
#     torch.save(model, f)

In [8]:
# # with open(MODEL_FILE_NAME.format(timestamp), 'rb') as f:
# # with open('models/trained_models/model-2019-05-22 18:36:17.481233.pt', 'rb') as f:
# with open('models/trained_models/model-2019-05-24 16:47:50.266977.pt', 'rb') as f:
#     model = torch.load(f)
#     # after load the rnn params are not a continuous chunk of memory
#     # this makes them a continuous chunk, and will speed up forward pass
#     model.rnn.flatten_parameters()

In [10]:
# val_loss = evaluate(model, corpus, criterion, device)

In [17]:
# /pytorch/aten/src/THC/THCTensorIndex.cu:362: void indexSelectLargeIndex(TensorInfo<T, IndexType>, 
# TensorInfo<T, IndexType>, TensorInfo<long, IndexType>, int, int, IndexType, IndexType, long) 
# [with T = float, IndexType = unsigned int, DstDim = 2, SrcDim = 2, IdxDim = -2, IndexIsMajor = true]: 
# block: [90,0,0], thread: [80,0,0] Assertion `srcIndex < srcSelectDimSize` failed.